<a href="https://colab.research.google.com/github/jjangmo91/GEE-PAM-Book/blob/main/4_2_2_%EB%AA%A8%EC%9E%90%EB%B0%98_%ED%83%90%EC%A7%80_%EC%A7%80%EC%88%98_NFAI%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q leafmap[raster]

In [ ]:
import ee, geemap
import requests, os
import xarray as xr
import numpy as np
import leafmap
import rioxarray
import matplotlib
from shapely.geometry import shape
from pathlib import Path
from datetime import datetime

# Earth Engine 인증
ee.Authenticate()
# Earth Engine 초기화
ee.Initialize(project='ee-') # 본인 계정

In [ ]:
# 관심지역
wdpa = ee.FeatureCollection("WCMC/WDPA/current/polygons")
aoi = wdpa.filter(ee.Filter.eq('WDPAID', 180)).first()

# 20km 버퍼
buffer_km = 20
geom_buf = aoi.geometry().buffer(buffer_km * 1000)

# bbox 계산(서, 동, 남, 북)
bounds = geom_buf.bounds().getInfo()
bbox_poly = shape(bounds)
west, south, east, north = bbox_poly.bounds
print("BBOX:", west, east, south, north)

In [ ]:
# NCSS (NetCDF Subset Service) request URL
NCSS_URL = "https://tds-odatis.aviso.altimetry.fr/thredds/ncss/grid/dataset-sargassum-cls-merged-msi-oli-global-lr"
# AVISO+ 인증
USER = "" # 본인 계정
PW = "" # 비밀번호

# 영역
BBOX = dict(north=north, south=south, east=east, west=west)

# 기간
YEAR, MONTH = 2024, 7
time_start = f"{YEAR}-{MONTH:02d}-01T00:00:00Z"
time_end = f"{YEAR}-{MONTH:02d}-31T23:59:59Z"

# 변수
VARS = ["nfai_mean","nfai_max","nfai_min","nfai_nbpts"]

In [ ]:
def fetch_ncss(url: str, out_path):
    """NCSS URL로 NetCDF 다운로드"""
    params = {
        "var": VARS,
        "north": BBOX["north"],
        "south": BBOX["south"],
        "east": BBOX["east"],
        "west": BBOX["west"],
        "horizStride": 1,
        "time_start": time_start,
        "time_end": time_end,
        "accept": "netcdf4",
    }

    with requests.Session() as s:
        s.auth = (USER, PW)
        r = s.get(url, params=params, timeout=180)
        r.raise_for_status()
        ctype = r.headers.get("Content-Type", "")
        if "netcdf" not in ctype.lower():
            raise RuntimeError(f"서버가NetCDF가아닌응답을반환했습니다: {ctype}")
        out_path.write_bytes(r.content)
    return out_path

In [ ]:
nc_path = Path(f"{YEAR}_{MONTH:02d}_nfai.nc")
saved = fetch_ncss(NCSS_URL, nc_path)
print(f"[저장완료] {saved}")

In [ ]:
# nc 열기
ds = xr.open_dataset(nc_path, engine="h5netcdf")
print("=== Dataset 개요===")
print(ds)

# 시간차원
time_dim = None
for cand in ["time", "Time", "t"]:
    if cand in ds.dims:
        time_dim = cand
        break

if time_dim:
    n_layers = ds.sizes[time_dim]
    print(f"\n이파일에는총{n_layers}개의시간레이어가있습니다.")
    print("시간목록:", ds[time_dim].values)
else:
    print("\n이파일에는시간차원이없습니다.")
ds.close()

In [ ]:
# 날짜
date_str = "2024-07-01"
date = np.datetime64(date_str)
# 변수
var_name = "nfai_mean"
with xr.open_dataset(nc_path, engine="h5netcdf") as ds:
    # 모자반이없는픽셀은-0.5 값으로설정
    nodata = float(ds[var_name].attrs.get("_FillValue", -0.5))
    da = (
        ds[var_name].sel(time=date)
        .where(lambda x: x != nodata)
        .rename(latitude="y", longitude="x")
        .sortby("y", ascending=False)
        .rio.write_crs("EPSG:4326")
        .rio.write_nodata(nodata)
        .fillna(nodata)
        .load()
    )
tif_path = Path(f"nfai_{date_str}_{datetime.now():%H%M%S}.tif")
da.rio.to_raster(tif_path)
print("saved:", tif_path)

In [ ]:
# vmin/vmax 계산
valid = da.where(da != nodata)
vmin = float(valid.quantile(0.02))
vmax = float(valid.quantile(0.98))

# 중심좌표
lat_c = float(da["y"].mean())
lon_c = float(da["x"].mean())

# 지도생성
m = leafmap.Map(width="700px", height="400px", center=(lat_c, lon_c), zoom=10)
m.add_basemap("Esri.WorldImagery")
m.add_raster(
    str(tif_path),
    colormap="viridis",
    vmin=vmin, vmax=vmax,
    nodata=nodata,
    opacity=1.0,
    layer_name=f"{var_name} {date_str}",
    zoom_to_layer=False,
)
m.add_colormap("viridis", vmin=vmin, vmax=vmax, label=var_name)
m

In [ ]:
!pip install -q leafmap[raster] xarray rioxarray netcdf4 h5netcdf

import ee, geemap
import requests
import xarray as xr
import numpy as np
from pathlib import Path
import leafmap
from shapely.geometry import shape

# Earth Engine 인증 및 초기화
ee.Authenticate()
ee.Initialize(project='') # 본인 계정

# AVISO 접속 정보
USER = "" # 본인 계정
PW = "" # 비밀번호

# 관심 지역(AOI) 설정
BBOX = {
    'north': 18.35,
    'south': 18.20,
    'east': -68.50,
    'west': -68.65
}

# 데이터셋 정보 수정
NCSS_URL = "https://tds-odatis.aviso.altimetry.fr/thredds/ncss/grid/dataset-sargassum-modis-global-hr"
VARS = ["nfai", "level2_flag"]

# 분석 기간 설정
YEAR, MONTH, DAY = 2022, 6, 15
time_start = f"{YEAR}-{MONTH:02d}-{DAY:02d}T00:00:00Z"
time_end = f"{YEAR}-{MONTH:02d}-{DAY:02d}T23:59:59Z"

In [ ]:
# 데이터 다운로드 함수 정의
def fetch_ncss(url: str, out_path, variables, time_start, time_end, bbox):
    params = {
        "var": variables, "north": bbox["north"], "south": bbox["south"],
        "east": bbox["east"], "west": bbox["west"], "horizStride": 1,
        "time_start": time_start, "time_end": time_end, "accept": "netcdf4",
    }
    with requests.Session() as s:
        s.auth = (USER, PW)
        r = s.get(url, params=params, timeout=300)
        r.raise_for_status()
        out_path.write_bytes(r.content)
    return out_path

In [ ]:
# 데이터 다운로드, 처리 및 시각화 실행
nc_path = Path("sargassum_parque_del_este.nc")
saved_nc = fetch_ncss(NCSS_URL, nc_path, VARS, time_start, time_end, BBOX)

tif_path = Path("sargassum_parque_del_este.tif")
with xr.open_dataset(nc_path, engine="h5netcdf") as ds:
    nfai_data = ds["nfai"]
    flag_data = ds["level2_flag"]
    nfai_masked = nfai_data.where(flag_data != 1)
    nfai_masked = nfai_masked.fillna(-9999)
    nfai_masked.rio.write_nodata(-9999, inplace=True)
    nfai_masked.rio.write_crs("EPSG:4326", inplace=True)
    nfai_masked.rio.to_raster(tif_path)

# 지도 시각화
center_lat = (BBOX["north"] + BBOX["south"]) / 2
center_lon = (BBOX["east"] + BBOX["west"]) / 2
m = leafmap.Map(center=(center_lat, center_lon), zoom=14)
m.add_basemap("Esri.WorldImagery")
m.add_raster(
    str(tif_path),
    indexes=[1],
    colormap="viridis",
    vmin=0, vmax=0.10,
     layer_name=f"Sargassum - Punta Cana ({YEAR}-{MONTH}-{DAY})"
)
display(m)